In [1]:
#Reference => sentdex tutorial on Youtube
!pip install gym #already installed

In [2]:
import gym
env = gym.make('MountainCar-v0') #for discription https://github.com/openai/gym/wiki/MountainCar-v0
env.reset()

array([-0.58779031,  0.        ])

In [3]:
import random
done = False

while not done:
  action = (int(random.random()*10))%3 #action 0->push car left; action 1->do nothing; action 2-> push car right
  state, reward, done, _ = env.step(action)# state is a tuple of (position, velocity)
  print(state)  

  env.render()

env.close()

#the car needs to build momentum

[-5.87311848e-01  4.78466374e-04]
[-0.58635844  0.00095341]
[-0.58493711  0.00142133]
[-0.58405833  0.00087878]
[-0.58272859  0.00132974]
[-0.58195769  0.00077089]
[-0.58075134  0.00120635]
[-0.58011844  0.0006329 ]
[-0.57906367  0.00105477]
[-5.78594835e-01  4.68838525e-04]
[-0.57671539  0.00187944]
[-0.57343926  0.00327613]
[-0.56979072  0.00364854]
[-0.56579685  0.00399388]
[-0.56048733  0.00530952]
[-0.55390171  0.00658562]
[-0.54708914  0.00681258]
[-0.54010053  0.00698861]
[-0.53398821  0.00611232]
[-0.52679798  0.00719023]
[-0.52058376  0.00621422]
[-0.51339215  0.00719161]
[-0.50527708  0.00811507]
[-0.49629935  0.00897773]
[-0.48652614  0.00977321]
[-0.4780304   0.00849574]
[-0.46887536  0.00915504]
[-0.45912891  0.00974646]
[-0.45086297  0.00826594]
[-0.44413823  0.00672474]
[-0.43900381  0.00513441]
[-0.43349707  0.00550674]
[-0.42965789  0.00383918]
[-0.42751398  0.00214391]
[-0.42608077  0.00143321]
[-0.42536855  0.00071222]
[-4.25382448e-01 -1.38935610e-05]
[-0.42612235 -

## Implementing Q Table

We will be implementing SARSAMAX

![](SARSAMAX.png)

- State contains continous values
- Q Table will contains prohibitively large number of state values
- We need to **dicretize them** in order to create Q Table
- Initially Q Table will contain random values

In [4]:
#display observation space low and high values
print(env.observation_space.high)#[position,velocity]
print(env.observation_space.low)#[position, velocity]
print(env.action_space.n)

[0.6  0.07]
[-1.2  -0.07]
3


In [5]:
DISCRETE_OS_SIZE = [20] * len(env.observation_space.high)#os->observation space; 20 is the bucket size chosen by us (hyperparameter)
DISCRETE_OS_SIZE # We will have 20 discrete combinations for position and velocity. 
#State space will be 20 * 20 = 400

[20, 20]

In [6]:
#For each of 20 discrete state, we will have a bucket containing some range of position values and some range of velocity value
discrete_os_bucket_size = (env.observation_space.high-env.observation_space.low)/DISCRETE_OS_SIZE
discrete_os_bucket_size

array([0.09 , 0.007])

In [9]:
#creating Q Table
import numpy as np
#Q Table is assigned arbitrarily

#q_table = np.random.uniform(low=-2,high=0,size=(DISCRETE_OS_SIZE + [3])) #uniform is used when we want random values drawn from [low,high) range

q_table = np.zeros(DISCRETE_OS_SIZE + [3])

#low and high are randomly chosen
print(q_table.shape)
q_table #Q Table is of 3 Dimension  poition_bucket (20) * velocity_bucket (20) * action (3)

(20, 20, 3)


array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

## Now we will implement SARSAMAX algorithm to fill the Q Table

In [10]:
LEARNING_RATE = 0.1 #hyperparameter 
DISCOUNT = 0.95 #hyperparameter; how important are future rewards  
EPISODES = 25000 #hyperparameter
RENDER_EVERY = 5000

#a helper function to discretize continous states as returned by the environment 
def get_discrete_state(state):
    #discrete_state -> floor((current_state- lowest_state)/bucket_size)
    discrete_state = (state - env.observation_space.low) / discrete_os_bucket_size
    return tuple(discrete_state.astype(np.int)) #taking the floor value

In [11]:
# Exploration settings
epsilon = 1  # not a constant, qoing to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)

In [12]:
for episode in range(EPISODES):
    done = False
    discrete_state = get_discrete_state(env.reset()) #we always start with initial discrete state
    if episode % RENDER_EVERY == 0:
        render=True
        print(episode)
    else:
        render=False
        
    while not done:
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(q_table[discrete_state])
        else:
            # Get random action
            action = np.random.randint(0, env.action_space.n)
        state, reward, done, _ = env.step(action)# state is a tuple of (position, velocity)
        if render:
            env.render()
        new_discrete_state = get_discrete_state(state)
        if not done:
            q_st_at=q_table[discrete_state+(action,)]
            q_st_plus_1_a = np.max(q_table[new_discrete_state])
            new_q_st_at = q_st_at + LEARNING_RATE * (reward + DISCOUNT * q_st_plus_1_a - q_st_at)
            q_table[discrete_state+(action,)] = new_q_st_at
        elif state[0] >= env.goal_position: #if the episode terminates because we have achieved our goal
            new_q_st_at = q_st_at + LEARNING_RATE * (0 + DISCOUNT * q_st_plus_1_a - q_st_at)
        discrete_state = new_discrete_state
     # Decaying is being done every episode if episode number is within decaying range
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value
env.close()

0
5000
10000


KeyboardInterrupt: 

In [ ]:
discrete_state+(action,)